In [1]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image


import sqlite3
import numpy as np
from PIL import Image
import os
import cv2

def draw_boundary(img,classifier,scaleFactor,minNeighbors,colors,text,clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)
    
    coords=[]
    
    for(x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),colors,2)
        id,pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
        
        if confidence>70:
            if id:
                if(int(record[id-1][4])==0):
                    cv2.putText(img, "SAFE", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255,0), 2, cv2.LINE_AA)
                    cv2.putText(img, "Name-"+str(record[id-1][1]), (x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255,0), 1, cv2.LINE_AA)
                    cv2.putText(img, "Age-"+str(record[id-1][2]), (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255,0), 1, cv2.LINE_AA)
                    cv2.putText(img, "Gender-"+str(record[id-1][3]), (x,y+h+90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255,0), 1, cv2.LINE_AA)
                    cv2.putText(img, "Criminal Records-"+str(record[id-1][4]), (x,y+h+120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255,0), 1, cv2.LINE_AA)
                else:
                    cv2.putText(img, "WARNING", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.putText(img, "Name-"+str(record[id-1][1]), (x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 1, cv2.LINE_AA)
                    cv2.putText(img, "Age-"+str(record[id-1][2]), (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 1, cv2.LINE_AA)
                    cv2.putText(img, "Gender-"+str(record[id-1][3]), (x,y+h+90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 1, cv2.LINE_AA)
                    cv2.putText(img, "Criminal Records-"+str(record[id-1][4]), (x,y+h+120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0,255), 1, cv2.LINE_AA)
        else:
            #cv2.putText(img, "Unknown", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            classifier =load_model('./Emotion_Detection.h5')
            class_labels = ['Angry','Happy','Neutral','Sad','Surprise']
            roi_gray = gray[y:y+h,x:x+w]
            roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
            if np.sum([roi_gray])!=0:
                roi = roi_gray.astype('float')/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

                preds = classifier.predict(roi)[0]
                #print("\nprediction = ",preds)
                label=class_labels[preds.argmax()]
                #print("\nprediction max = ",preds.argmax())
                #print("\nlabel = ",label)
                label_position = (x,y)
                if(label=="Angry"):
                    label="Suspicious(Angry)"
                    cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),3)
                cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
            else:
                cv2.putText(img,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
            print("\n\n")
            
            coords=[x,y,w,h]
        return coords
def recognize(img,clf,faceCascade):
    coords = draw_boundary(img,faceCascade,1.1,10,(0,255,0),"Face",clf)
    
    
    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
print(dir (cv2.face))
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

#-------------------------------FetchingData---------------------------------
db_connect=sqlite3.connect("Facebase.db")
fetch_query = "SELECT * FROM People;"
cur=db_connect.cursor()
cur.execute(fetch_query)
record=cur.fetchall()
record.sort()


#--------------------------------Fetching End-------------------------------

video_capture = cv2.VideoCapture(0)
while True:
    ret,img = video_capture.read()
    img = recognize(img,clf,faceCascade)
    cv2.imshow("Criminal Face Identification",img)
    #cv2.imshow()
    
    if cv2.waitKey(1)==13:
        break

video_capture.release()
cv2.destroyAllWindows()


['BIF_create', 'EigenFaceRecognizer_create', 'FisherFaceRecognizer_create', 'LBPHFaceRecognizer_create', 'MACE_create', 'MACE_load', 'StandardCollector_create', '__doc__', '__loader__', '__name__', '__package__', '__spec__', 'createFacemarkAAM', 'createFacemarkKazemi', 'createFacemarkLBF', 'drawFacemarks', 'getFacesHAAR', 'loadDatasetList', 'loadFacePoints', 'loadTrainingData']


























































































































































































